# Salary Predictions Based on Job Descriptions

In [1]:
#load the data into a Pandas dataframe

In [2]:
#look for duplicate data, invalid data, or corrupt data and remove it

In [3]:
#summarize each feature variable
#summarize the target variable
#look for correlation between each feature and the target
#look for correlation between features

In [4]:
#create an extremely simple model and measure its efficacy (average of each industry)

Brainstorm 3 models that you think may improve results over the baseline model based on your EDA and explain why they're reasonable solutions here.

Also write down any new features that you think you should try adding to the model based on your EDA, e.g. interaction variables, summary statistics for each group, etc